In [ ]:
    # Import libraries.
    import urllib
    import requests
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    import time
    import numpy as np
    import math
    import random
    API_Key = "Insert API KEY"

In [ ]:
#Retrieve the input data for the problem and store them in variables as dataframes.

df_carros = pd.read_excel(r'C:\Users\lfxao\Documents\Jupyter\Instancias V8_carros.xlsx')
df_ocorrencias = pd.read_excel(r'C:\Users\lfxao\Documents\Jupyter\Instancias V8_ocorrencias.xlsx')
df_atendimento = pd.read_excel(r'C:\Users\lfxao\Documents\Jupyter\Instancias V8_tempos.xlsx')
df_disponibilidade = pd.read_excel(r'C:\Users\lfxao\Documents\Jupyter\Instancias V8_disponibilidade.xlsx')
df = pd.read_excel(r'C:\Users\lfxao\Documents\Jupyter\Instancias V8.xlsx')
df1 = df

with open("matriz8.txt", "r") as arquivo:
    M = arquivo.read()
matriz = eval(M)

In [ ]:
# Initialize the Solution Matrix with the sequence of service locations.
def ini_solucao_seq_atendimento(df_carros): 
    solucao_seq_atendimento = []
    linha1 = []
    for i in range(len(df_carros)):
        list.append(linha1, [])
    solucao_seq_atendimento = solucao_seq_atendimento + linha1     
    return (solucao_seq_atendimento)

# Initialize the Solution Matrix with the service times.
def ini_solucao_t_atendimento(df_carros): 
    solucao_t_atendimento = []
    linha2 = []
    for i in range(len(df_carros)):
        list.append(linha2, [])
    solucao_t_atendimento = solucao_t_atendimento + linha2     
    return (solucao_t_atendimento)

#Initialize the vector of occurrence cardinality: 0 -> not served / 1 -> served.
def ini_card_ocorrencias(df_ocorrencias):
    card_ocorrencias = []
    for i in df_ocorrencias['Unid']:
        atendido = 0
        card_ocorrencias.append(atendido)
    return (card_ocorrencias)

#Initialize the vector of occurrence priorities based on the input data.
def ini_prio_ocorrencias(df_atendimento):
    prio_ocorrencias = []
    for i, p in enumerate (df_atendimento['Prioridade']):
        prioridade = df_atendimento.loc[i, "Prioridade"]
        prio_ocorrencias.append(prioridade)
    return (prio_ocorrencias)

#Initialize the vector of occurrence durations based on the input data.
def ini_duracao(df_atendimento):
    duracao = []
    for i, d in enumerate (df_atendimento['Duração']):
        du = df_atendimento.loc[i, "Duração"]
        duracao.append(du)
    return (duracao)

#Initialize the initial position of the service units in the Displacement Matrix.
def ini_pos_k(df_disponibilidade):
    pos_k = []
    for i, p in enumerate (df_disponibilidade['Unid']):
        posicao = i
        pos_k.append(posicao)
    return (pos_k)

#Initialize the vector of service unit availability based on the input data.
def ini_v_disponib_k(df_disponibilidade):
    v_disponib_k = []
    for i, d in enumerate (df_disponibilidade['Diponib']):
        disponibilidade = df_disponibilidade.loc[i, 'Diponib']
        v_disponib_k.append(disponibilidade)
    return (v_disponib_k)

#Create a list of possible occurrences to be served.
def gerar_candidatos(v_disponib_k, duracao, prio_ocorrencias, pos_k, card_ocorrencias, matriz):
    candidatos = []
    for i in range(len(v_disponib_k)):
        c = i + 1
        disponibilidade = v_disponib_k[i]
        linha =[]
        for j in range(len(duracao)):
            o = j + 1
            dur = duracao[j]
            prio = prio_ocorrencias[j]
            desloc = matriz[pos_k[i]][j + len(v_disponib_k)]
            t = round(disponibilidade + dur + desloc, 1)
            if card_ocorrencias[j] == 0:
                if prio == min(prio_ocorrencias):
                    list.append(linha, [c, o, t])
        candidatos = candidatos + linha
    return (candidatos)

#Sort the Candidates Matrix.
def ordenar_candidatos(candidatos):
    candidatos_ordem = np.array(candidatos)
    candidatos_ordem = candidatos_ordem[candidatos_ordem[:,2].argsort()]
    candidatos_ordem = candidatos_ordem.tolist()
    for i in candidatos_ordem:
        i[0] = int(i[0])
        i[1] = int(i[1])
        i[2] = float(i[2])
    return (candidatos_ordem)
    
#Choose the candidate with an alpha degree of randomness.
def escolher_candidado_alfa(candidatos, alfa, escolha_min, df_carros):
    if len(candidatos) == len(df_carros):
        escolha = candidatos_ordem[0]
        return (escolha)
    else:
        tam_cand = len(candidatos_ordem)-1
        max_cand = math.floor(tam_cand * alfa)
        if max_cand < escolha_min:
            max_cand = escolha_min
        esc = random.randint(0,max_cand)
        escolha = candidatos_ordem[esc]
        return (escolha)

#Choose the candidate with the lowest completion time.
def escolher_candidato(candidatos):
    t_menor = 1000
    for i in candidatos:
        if i[2] < t_menor:
            t_menor = i[2]
            escolha = i
    return (escolha)

#Add the chosen candidate in the Solution Matrix with the sequence of service locations.
def atu_solucao_seq_atendimento(escolha):
    solucao_seq_atendimento[escolha[0]-1].append(escolha[1])
    return (solucao_seq_atendimento)

#Add the chosen candidate in the Solution Matrix with the service time.
def atu_solucao_t_atendimento(escolha):
    solucao_t_atendimento[escolha[0]-1].append(escolha[2])
    return (solucao_t_atendimento)


#Update the value of the solution.
def atu_valor_da_solucao(solucao_t_atendimento):
    valor_da_solucao = 0
    for i in solucao_t_atendimento:
        if len(i) > 0:
            a = i[-1]
            if a > valor_da_solucao:
                valor_da_solucao = a
        else:
            a = 0
            valor_da_solucao += a
    return (valor_da_solucao)


#Update the vector of occurrence cardinality.
def atu_card_ocorrencias(escolha):
    card_ocorrencias[escolha[1]-1] = 1
    return (card_ocorrencias)

#Update the vector of occurrence priorities.
def atu_prio_ocorrencias(escolha):
    prio_ocorrencias[escolha[1]-1] = 100
    return(prio_ocorrencias)

#Update the vector of service unit positions.
def atu_pos_k(escolha, v_disponib_k):
    pos_k[escolha[0]-1] = len(v_disponib_k) + escolha[1]-1
    return (pos_k)

#Update the vector of service unit availability.
def atu_v_disponib_k(escolha, v_disponib_k):
    v_disponib_k[escolha[0]-1] = escolha[2]
    return (v_disponib_k)



In [ ]:
t0 = time.time()
n = 1
cont = 0
best_valor_da_solucao = 999999
for i in range(n):
    #Initialize the input variables.
    solucao_seq_atendimento = ini_solucao_seq_atendimento(df_carros)
    solucao_t_atendimento = ini_solucao_t_atendimento(df_carros)
    card_ocorrencias = ini_card_ocorrencias(df_ocorrencias)
    prio_ocorrencias = ini_prio_ocorrencias(df_atendimento)
    duracao = ini_duracao(df_atendimento)
    pos_k = ini_pos_k(df_disponibilidade)
    v_disponib_k = ini_v_disponib_k(df_disponibilidade)
    valor_da_solucao = 0
    #Loop to enumerate the candidates and put them in the Solution Matrix.
    for i in range (len(card_ocorrencias)):
        candidatos = gerar_candidatos(v_disponib_k, duracao, prio_ocorrencias, pos_k, card_ocorrencias, matriz)
        escolha = escolher_candidato(candidatos)
        solucao_seq_atendimento = atu_solucao_seq_atendimento(escolha)
        solucao_t_atendimento = atu_solucao_t_atendimento(escolha)
        card_ocorrencias = atu_card_ocorrencias(escolha)
        prio_ocorrencias = atu_prio_ocorrencias(escolha)
        pos_k = atu_pos_k(escolha, v_disponib_k)
        v_disponib_k = atu_v_disponib_k(escolha, v_disponib_k)
    valor_da_solucao = atu_valor_da_solucao(solucao_t_atendimento)
    if valor_da_solucao < best_valor_da_solucao:
        cont += 1
        best_solucao_seq_atendimento = solucao_seq_atendimento
        best_solucao_t_atendimento = solucao_t_atendimento
        best_solucao_t_chegada = solucao_t_chegada
        best_valor_da_solucao = valor_da_solucao
        
print('Solução Gulosa:')
print(best_solucao_seq_atendimento)
print(best_solucao_t_atendimento)
print(round(best_valor_da_solucao,1))
print(cont)
#print(card_ocorrencias)
#print(prio_ocorrencias)
#print(pos_k)
#print(v_disponib_k)
tf = time.time()
print(f'A execução demorou {(tf-t0):.2f}s')

In [ ]:
t0 = t1 = time.time()
n = 4612
cont = 0
sem_melhora = 0 
best_valor_da_solucao = 999999
alfa_ini = 0.2
alfa = alfa_ini
alfa_max = 0.5
max_it_sem_melhora = 1038
tx_sem_melhora = 0.1
continua = True
it = 0
i = 0
while continua:
    if t1 - t0 > 60:
        print('**** Time exceeds 60 seconds. ****')
        break
    #Iniciar as variáveis de entrada
    solucao_seq_atendimento = ini_solucao_seq_atendimento(df_carros)
    solucao_t_atendimento = ini_solucao_t_atendimento(df_carros)
    card_ocorrencias = ini_card_ocorrencias(df_ocorrencias)
    prio_ocorrencias = ini_prio_ocorrencias(df_atendimento)
    duracao = ini_duracao(df_atendimento)
    pos_k = ini_pos_k(df_disponibilidade)
    v_disponib_k = ini_v_disponib_k(df_disponibilidade)
    valor_da_solucao = 0
    #Loop para elencar os candidatos e colocar na matriz de solução
    for j in range (len(card_ocorrencias)):
        candidatos = gerar_candidatos(v_disponib_k, duracao, prio_ocorrencias, pos_k, card_ocorrencias, matriz)
        candidatos_ordem = ordenar_candidatos(candidatos)
        escolha = escolher_candidado_alfa(candidatos_ordem, alfa, 0, df_carros)
        solucao_seq_atendimento = atu_solucao_seq_atendimento(escolha)
        solucao_t_atendimento = atu_solucao_t_atendimento(escolha)
        card_ocorrencias = atu_card_ocorrencias(escolha)
        prio_ocorrencias = atu_prio_ocorrencias(escolha)
        pos_k = atu_pos_k(escolha, v_disponib_k)
        v_disponib_k = atu_v_disponib_k(escolha, v_disponib_k)
    valor_da_solucao = atu_valor_da_solucao(solucao_t_atendimento)
    if valor_da_solucao < best_valor_da_solucao:
        print(i, it, alfa, valor_da_solucao)
        iteracao = i 
        cont += 1
        best_solucao_seq_atendimento = solucao_seq_atendimento
        best_solucao_t_atendimento = solucao_t_atendimento
        best_valor_da_solucao = valor_da_solucao
        alfa = alfa_ini
        it = 0
    else:
        sem_melhora +=1
    if sem_melhora > max_it_sem_melhora:
        alfa = min(alfa_max, alfa + tx_sem_melhora)
        print(alfa, it)
        sem_melhora = 0
    if it > n:
        continua = False
    it += 1
    i += 1
    t1 = time.time()
print('Solução Gulosa Alfa Dinâmico:')
print('---'*40)
print(f'A matriz com a sequencia de atendimentos é dada por: {best_solucao_seq_atendimento}')
print(f'A matriz com os tempos de atendimentos é dada por: {best_solucao_t_atendimento}')
print(f'O Valor da melhor solução é: {best_valor_da_solucao:.1f}')
print(f'Ao longo do processo a solução foi melhorada {cont} vezes')
print(f'A melhor solução foi encontrada na {iteracao}ª iteração')
#print(card_ocorrencias)
#print(prio_ocorrencias)
#print(pos_k)
#print(v_disponib_k)
tf = time.time()
print(f'A execução demorou {(tf-t0):.2f}s')